# `formerly test_simulation_v6.ipynb`

In [1]:
import multiprocessing as mp
import pandas as pd
import sonetsim
import os

num_logical_processors = mp.cpu_count()
pd.set_option("display.max_columns", 25)
pd.set_option("display.width", 100)

/home/cason/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Note: to be able to use all crisp methods, you need to install some additional packages:  {'graph_tool'}


Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw'}


In [2]:
num_logical_processors

16

In [3]:
def set_num_logical_processors(n):
    global num_logical_processors
    num_logical_processors = n

In [4]:
homophilies = [x / 4 for x in range(1, 2)]
isolations = [x / 4 for x in range(1, 2)]
insulations = [x / 4 for x in range(1, 2)]
affinities = [x / 4 for x in range(1, 2)]
num_nodes = [50 * (x + 1) ** 2 for x in range(0, 2)]
edge_multipliers = [2 * x for x in range(1, 2)]
num_communities = [2 * x**2 for x in range(1, 2)]
# resloutions      = [x for x in range(0,2)]


def create_simulation_and_evaluation_grid_search(seed):
    grid_search = []
    for hom in homophilies:
        for iso in isolations:
            for ins in insulations:
                for aff in affinities:
                    for nns in num_nodes:
                        for em in edge_multipliers:
                            nes = int(nns * em)
                            for ncs in num_communities:
                                grid_search.append(
                                    [hom, iso, ins, aff, nns, nes, ncs, seed]
                                )
    return grid_search

In [13]:
index_cols = [
    "algorithm",
    "community",
    "target_homophily",
    "target_isolation",
    "target_insulation",
    "target_affinity",
    "num_nodes",
    "num_edges",
    "edge_multiplier",
    "num_communities",
    "resolution",
    "seed",
]
# num_batches = 111
num_batches = 2
# eval_dir = "/workspace/research/Funded/Ethical_Reccomendations/Paper_Poster/ACM/EchoChambers/FigTeX/EX-Networks/Method/JUPYT/Simulator/soscsim/checkpoints_v2/mega_eval_df/"
eval_dir = "/mnt/r/Funded/Ethical_Reccomendations/Paper_Poster/ACM/EchoChambers/FigTeX/EX-Networks/Method/JUPYT/Simulator/soscsim/checkpoints_v2/mega_eval_df/"

In [ ]:
for seed in range(0, 5, 1):
    print(f"working on seed {seed}")
    grid_search = create_simulation_and_evaluation_grid_search(seed)
    _grid_search = create_simulation_and_evaluation_grid_search(seed)
    chunk_size = len(grid_search) / num_batches

    for n in range(num_batches):
        if os.path.isfile(eval_dir + f"seed_{seed}_batch_{n}.parquet"):
            print(f"skipping seed {seed} batch {n}, already processed")
            print("------------------------------------\n")
            continue
        print(f"working on seed {seed} batch {n}")
        batch = grid_search[int(chunk_size * n) : int(chunk_size * (n + 1))].copy()

        print(f"working on seed {seed} batch {n} graph simulations")
        with mp.Pool(num_logical_processors) as pool:
            graphs = pool.starmap(func=sonetsim.simulation, iterable=batch)

        _batch = _grid_search[int(chunk_size * n) : int(chunk_size * (n + 1))]
        params = []
        for param_set, graph in zip(_batch, graphs):
            params.append(param_set + list(graph))

        print(f"working on seed {seed} batch {n} parallel algorithms")
        with mp.Pool(num_logical_processors) as pool:
            evals = pool.starmap(
                func=sonetsim.evaluate_communities_parallel, iterable=params
            )

        print(f"working on seed {seed} batch {n} serial algorithms")
        for param_set in params:
            evals.append(sonetsim.evaluate_communities_serial(*param_set))

        print(f"working on seed {seed} batch {n} saving")
        mega_eval_df = pd.concat(evals)
        mega_eval_df.to_parquet(eval_dir + f"seed_{seed}_batch_{n}.parquet")
        print("------------------------------------\n")
    print("----------------------------------------------\n")

In [16]:
os.path.isfile(eval_dir + f"seed_{seed}_batch_{n}.parquet")

False

In [18]:
(eval_dir + f"seed_{seed}_batch_{n}.parquet")

'/mnt/r/Funded/Ethical_Reccomendations/Paper_Poster/ACM/EchoChambers/FigTeX/EX-Networks/Method/JUPYT/Simulator/soscsim/checkpoints_v2/mega_eval_df/seed_0_batch_0.parquet'